In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, HTML

# ------------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------------
ROOT_FOLDER = r"C:\Users\FahdRahman\IONITY\Operations - Site Documentation - Documents"

DOC_FOLDERS = [
    "01_Local_Documentation",
    "02_Electric_Organization",
    "03_Electric_Documentation",
    "04_Operation_Service_Maintenance",
    "06_Risk_Assessments"
]

# Folders to ignore at the top-level
IGNORE_FOLDERS = [
    "Contractor_HSSE",
    "Templates",
    "Site_Folder_Structure_Fahd",
    "Site_XX",
    "Site_XX (NEW)",
    "Site_XX (T&R)",
    "Site_XX REWORKED BY HENRIK",
    "ZZ_CMG Owner"
]

# ------------------------------------------------------------------
# Build Country -> Site mapping (with total document count)
# ------------------------------------------------------------------
countries = {}
site_doc_count = {}  # store total documents per site

for country in os.listdir(ROOT_FOLDER):
    country_path = os.path.join(ROOT_FOLDER, country)
    if not os.path.isdir(country_path):
        continue
    if country in IGNORE_FOLDERS or country.startswith("Site_"):
        continue  # skip unwanted folders

    sites_dict = {}
    for site in os.listdir(country_path):
        site_path = os.path.join(country_path, site)
        if not os.path.isdir(site_path):
            continue

        # Count total documents in all DOC_FOLDERS
        total_docs = 0
        for folder in DOC_FOLDERS:
            folder_path = os.path.join(site_path, folder)
            if os.path.isdir(folder_path):
                total_docs += len(os.listdir(folder_path))

        sites_dict[site] = total_docs
        site_doc_count[(country, site)] = total_docs

    if sites_dict:
        countries[country] = sites_dict

# ------------------------------------------------------------------
# UI Widgets
# ------------------------------------------------------------------
country_dropdown = widgets.Dropdown(
    options=["Select a country"] + list(countries.keys()),
    description="Country:",
    style={'description_width': '120px'},
    layout=widgets.Layout(width="400px")
)

site_dropdown = widgets.Dropdown(
    options=["Select a site"],
    description="Site:",
    style={'description_width': '120px'},
    layout=widgets.Layout(width="400px")
)

output_area = widgets.Output()

# ------------------------------------------------------------------
# Update site dropdown when a country is selected
# ------------------------------------------------------------------
def update_sites(change):
    output_area.clear_output()
    if change["new"] == "Select a country":
        site_dropdown.options = ["Select a site"]
        return

    selected_country = change["new"]
    sites_dict = countries[selected_country]
    
    # Include total documents in brackets
    site_options = ["Select a site"] + [
        f"{site} ({doc_count} docs)" for site, doc_count in sites_dict.items()
    ]
    site_dropdown.options = site_options

country_dropdown.observe(update_sites, names="value")

# ------------------------------------------------------------------
# Display site details when selected
# ------------------------------------------------------------------
def show_site_details(change):
    output_area.clear_output()
    if change["new"] == "Select a site":
        return

    selected_country = country_dropdown.value
    # Extract site name before the bracket
    selected_site = change["new"].split(" (")[0]
    site_path = os.path.join(ROOT_FOLDER, selected_country, selected_site)

    with output_area:
        print(f"\nSite: {selected_site}")
        print(f"Country: {selected_country}")
        print("──────────────────────────────────────────\n")

        # Create accordion for folders
        folder_items = []
        titles = []

        for folder in DOC_FOLDERS:
            folder_path = os.path.join(site_path, folder)
            if os.path.isdir(folder_path):
                files = os.listdir(folder_path)
            else:
                files = []

            file_text = "<br>".join(files) if files else "<i>No files found</i>"
            folder_items.append(widgets.HTML(f"<b>Files in {folder}:</b><br>{file_text}"))
            titles.append(folder)

        accordion = widgets.Accordion(children=folder_items)
        for i, t in enumerate(titles):
            accordion.set_title(i, t)

        display(accordion)

site_dropdown.observe(show_site_details, names="value")

# ------------------------------------------------------------------
# DISPLAY UI (ready for Voilà)
# ------------------------------------------------------------------
ui = widgets.VBox([country_dropdown, site_dropdown, output_area])
display(ui)
